In [33]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
import matplotlib
import pandas as pd
import pickle
from utils.post_processing_utils import *
import seaborn as sns
from hernando_behavioural_analysis_dev.BehaviouralAnalysis.utils import custom_functions as cuf
from scipy import stats
from utils.plotting import calculate_error_bars

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
def moving_average(x, window=50):
    rolling_average = np.empty(len(x))
    rolling_average[:] = np.nan
    for i in range(window, len(x)):
        win = range((i - window), i)
        rolling_average[i] = np.mean(x[win])
    return rolling_average

In [35]:
site = 'tail'
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\state_change_data')
state_change_data_file = os.path.join(processed_data_dir,  'state_change_data_{}_mice.csv'.format(site))
all_session_change_data = pd.read_pickle(state_change_data_file)

In [36]:
np.where(np.array(fiber_options) != experiment_to_process['fiber_side'].values[0])[0][0] + 1

1

In [58]:

pre_pc = []
post_pc = []
mice = [ 'SNL_photo21', 'SNL_photo22', 'SNL_photo26', 'SNL_photo37', 'SNL_photo43','SNL_photo70', 'SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
moving_avs = []
#mice = ['SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
for mouse_num, mouse_id in enumerate(mice):
    state_change_data = {}
    exp_type = 'state change white noise'
    all_experiments = get_all_experimental_records()
    all_experiments = remove_bad_recordings(all_experiments)
    experiment_to_process = all_experiments[(all_experiments['experiment_notes'] == exp_type) & (all_experiments['mouse_id'] == mouse_id)]
    contra_side = experiment_to_process['fiber_side'].values[0]
    fiber_options = np.array(['left', 'right'])
    contra_choice_ind = (np.where(fiber_options != contra_side)[0] + 1)[0]
    if contra_choice_ind == 2:
        fiber_side_numeric = 7
    else:
        fiber_side_numeric = 1
    print(contra_side,fiber_side_numeric)
    traces, trial_data = open_experiment(experiment_to_process)
    trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
    only_contra = trial_data.loc[trial_data['Trial type'] == fiber_side_numeric] 
    red_trial_data = only_contra[only_contra['State name'] == 'TrialStart']
    post_trials = red_trial_data[np.logical_and(red_trial_data['Trial num'] >= 150, red_trial_data['Trial num'] <= 200)]
    post_trials = post_trials[post_trials['Trial type'] == fiber_side_numeric]
    wn_correct_trials =  post_trials[(post_trials['Response'] == contra_choice_ind) & (post_trials['Trial outcome'] == 1)] 
    wn_trial_type = wn_correct_trials['Trial type'].unique()[0]
    wn_trials = post_trials[post_trials['Trial type'] == wn_trial_type]
    post_pc.append(np.mean(wn_trials['Trial outcome'].values) * 100)
    pre_pc.append(np.mean(red_trial_data[np.logical_and(red_trial_data['Trial num'] < 150, red_trial_data['Trial num'] >= 100)]['Trial outcome'].values) * 100)
    moving_avs.append(moving_average(red_trial_data['Trial outcome'].values, window=20))

Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683, 684, 692, 700, 783, 794, 796, 797, 813, 829,
            842, 843, 844, 845, 850, 853],
           dtype='int64')
left 7
Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683, 684, 692, 700, 783, 794, 796, 797, 813, 829,
            842, 843, 844, 845, 850, 853],
           dtype='int64')
right 1
Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683, 684, 692, 700, 783, 794, 796, 797, 813, 829,
            842, 843, 844, 845, 850, 853],
           dtype='int64')
right 1
Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683, 684, 692, 700, 783, 794, 796, 797, 813, 829,
            842, 843, 844, 845, 850, 853],
           dtype='int64')
right 1
Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,


In [50]:

moving_avs = []
response_times = []
missed_trials = []
performance = []

processed_data_dir = os.path.join('T:\\photometry_2AC\\processed_data\\state_change_data')
mice =  [ 'SNL_photo21', 'SNL_photo22', 'SNL_photo26', 'SNL_photo37', 'SNL_photo43','SNL_photo70', 'SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
#[ 'SNL_photo21', 'SNL_photo22', 'SNL_photo26', 'SNL_photo37', 'SNL_photo43', 'SNL_photo70', 'SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
moving_avs = []
#mice = ['SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
for mouse_num, mouse_id in enumerate(mice):
    state_change_data = {}
    exp_type = 'state change white noise'
    all_experiments = get_all_experimental_records()
    all_experiments = remove_bad_recordings(all_experiments)
    experiment_to_process = all_experiments[(all_experiments['experiment_notes'] == exp_type) & (all_experiments['mouse_id'] == mouse_id)]
    contra_side = experiment_to_process['fiber_side'].values[0]
    fiber_options = np.array(['right', 'left'])
    fiber_side_numeric = (np.where(fiber_options != contra_side)[0] + 1)[0]
    if fiber_side_numeric == 2:
        side = 1
    else:
        side = -1
    traces, trial_data = open_experiment(experiment_to_process)
    trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
    red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
    red_trial_data_for_missed_trials = trial_data[trial_data['State name'] == 'TrialStart']
    red_trial_data_for_missed_trials.loc[trial_data['Trial outcome'] == 1, 'Trial outcome'] = 0
    red_trial_data_for_missed_trials.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 1
    red_trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
    response_trial_data = trial_data[trial_data['State name'] == 'WaitForResponse']
    post_trials = red_trial_data[np.logical_and(red_trial_data['Trial num'] >= 150, red_trial_data['Trial num'] <= 200)]
    pre_trials = red_trial_data[np.logical_and(red_trial_data['Trial num'] < 150, red_trial_data['Trial num'] >= 100)]
    moving_avs.append(cuf.CalculateRBiasWindow(red_trial_data['First response'].reset_index(drop=True), red_trial_data['First choice correct'].reset_index(drop=True), 20) *side *100)
    response_times.append(moving_average(response_trial_data['Time end'].values - response_trial_data['Time start'].values, window=20))
    missed_trials.append(moving_average(red_trial_data_for_missed_trials['Trial outcome'].values, window=20) * 100)
    performance.append(moving_average(red_trial_data['Trial outcome'].values, window=20) * 100)




Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683, 684, 692, 700, 783, 794, 796, 797, 813, 829,
            842, 843, 844, 845, 850, 853],
           dtype='int64')
Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683, 684, 692, 700, 783, 794, 796, 797, 813, 829,
            842, 843, 844, 845, 850, 853],
           dtype='int64')
Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683, 684, 692, 700, 783, 794, 796, 797, 813, 829,
            842, 843, 844, 845, 850, 853],
           dtype='int64')
Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683, 684, 692, 700, 783, 794, 796, 797, 813, 829,
            842, 843, 844, 845, 850, 853],
           dtype='int64')
Int64Index([ 57,  98, 101, 213, 217, 218, 221, 223, 225, 226, 519, 559, 574,
            600, 603, 656, 683,

In [51]:
post_trials

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
1734,152.0,7.0,1.0,1.0,1.0,TrialStart,678.5749,678.5849,NaN,1.0,1.0,678.5749,692.8774,0.0,NaN,0.0,0.0
1741,153.0,7.0,1.0,1.0,1.0,TrialStart,693.0351,693.0451,1.0,NaN,3.0,693.0351,700.3744,0.0,1.0,0.0,0.0
1774,157.0,7.0,1.0,1.0,1.0,TrialStart,715.8215,715.8315,1.0,1.0,1.0,715.8215,722.2132,0.0,1.0,0.0,0.0
1783,158.0,7.0,1.0,1.0,1.0,TrialStart,722.3670,722.3770,1.0,1.0,0.0,722.3670,729.7000,0.0,1.0,0.0,0.0
1790,159.0,7.0,1.0,1.0,1.0,TrialStart,729.8428,729.8528,1.0,1.0,0.0,729.8428,737.2045,0.0,1.0,0.0,0.0
1799,160.0,7.0,1.0,1.0,1.0,TrialStart,737.3420,737.3520,2.0,1.0,0.0,737.3420,740.5792,1.0,2.0,0.0,1.0
1809,161.0,7.0,1.0,1.0,1.0,TrialStart,740.7137,740.7237,1.0,2.0,1.0,740.7137,752.0832,0.0,1.0,0.0,0.0
1879,167.0,7.0,1.0,1.0,1.0,TrialStart,773.0031,773.0131,2.0,1.0,1.0,773.0031,776.1562,1.0,2.0,0.0,1.0
1887,168.0,7.0,1.0,1.0,1.0,TrialStart,776.2919,776.3019,1.0,2.0,1.0,776.2919,783.0796,0.0,1.0,0.0,0.0
1925,172.0,7.0,1.0,1.0,1.0,TrialStart,793.1399,793.1499,1.0,1.0,1.0,793.1399,798.3807,0.0,1.0,0.0,0.0


In [59]:
post_pc

[60.0,
 92.3076923076923,
 58.620689655172406,
 86.20689655172413,
 84.61538461538461,
 70.96774193548387,
 34.48275862068966,
 86.66666666666667,
 54.54545454545454,
 55.00000000000001,
 53.84615384615385,
 42.10526315789473,
 24.0]

In [40]:
trial_data[trial_data['Trial outcome']==3]

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct


In [60]:
def plot_rolling_mean(var, mice, moving_avs, y_axis_label='% bias to big reward side'):
    
    all_mice, min_num_trials = align_mulitple_mice_moving_avs(mice, moving_avs)
    error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
                                                                     all_mice,
                                                                    error_bar_method='sem')
    font = {'size': 8}
    matplotlib.rc('font', **font)
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['font.sans-serif'] = 'Arial'
    matplotlib.rcParams['font.family']


    fig, axs = plt.subplots(1, 1, figsize=[2.5, 2])
    [axs.plot(m[:min_num_trials], alpha=0.5, c='gray', lw=0.5) for m in moving_avs]
    axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)
    axs.fill_between(np.arange(0, min_num_trials), error_bar_lower, error_bar_upper, alpha=0.5,
                                facecolor='#7FB5B5', linewidth=0)

    axs.axvline(150, color='k')
    axs.set_xlabel('trial number')
    axs.set_ylabel(y_axis_label)
    axs.spines['right'].set_visible(False)
    axs.spines['top'].set_visible(False)
    plt.tight_layout()
    figure_dir = r'T:\paper\revisions\state change behaviour'
    plt.tight_layout()
    plt.savefig(os.path.join(figure_dir, '{} developing over trials.pdf'.format(var)))

    plt.show()

In [20]:
def calc_per_mouse_mean(mouse_moving_avs):
    num_sessions = len(mouse_moving_avs)
    num_trials = [len(m) for m in mouse_moving_avs]
    min_num_trials = min(num_trials)
    all_sessions = np.empty((num_sessions, min_num_trials))
    all_sessions[:] = np.nan
    for i, session_data in enumerate(mouse_moving_avs):
        all_sessions[i, :min_num_trials] = session_data[:min_num_trials]
    moving_av = np.nanmean(all_sessions, axis=0)
    return moving_av

In [21]:
def align_mulitple_mice_moving_avs(mice, moving_avs):
    num_mice = len(mice)
    num_trials = [len(m) for m in moving_avs]
    min_num_trials = min([len(m) for m in moving_avs])
    all_mice = np.empty((num_mice, min_num_trials))
    all_mice[:] = np.nan
    for i, mouse_data in enumerate(moving_avs):
        all_mice[i, :min_num_trials] = mouse_data[:min_num_trials]
    return all_mice, min_num_trials

In [22]:
plot_rolling_mean('bias', mice, moving_avs, y_axis_label='% bias to ipsi side')

C:\Users\francescag\AppData\Local\Temp\ipykernel_22860\2871035215.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_22860\2871035215.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)


In [23]:
plot_rolling_mean('response time', mice, response_times, y_axis_label='response time (s)')


C:\Users\francescag\AppData\Local\Temp\ipykernel_22860\2871035215.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_22860\2871035215.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)


In [24]:
plot_rolling_mean('performance', mice, performance, y_axis_label='% correct')


C:\Users\francescag\AppData\Local\Temp\ipykernel_22860\2871035215.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_22860\2871035215.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)


In [28]:
plot_rolling_mean('missed trials', mice, missed_trials, y_axis_label='% missed trials')


C:\Users\francescag\AppData\Local\Temp\ipykernel_22860\2871035215.py:4: RuntimeWarning: Mean of empty slice
  error_bar_lower, error_bar_upper = calculate_error_bars(np.nanmean(all_mice, axis=0),
C:\Users\francescag\AppData\Local\Temp\ipykernel_22860\2871035215.py:16: RuntimeWarning: Mean of empty slice
  axs.plot(np.nanmean(all_mice,axis=0), c='#5e8c89', lw=1)


In [5]:
num_mice = len(mice)
num_trials = [len(m) for m in moving_avs]
max_num_trials = max([len(m) for m in moving_avs])
all_mice = np.empty((num_mice, max_num_trials))
all_mice[:] = np.nan
for i, mouse_data in enumerate(moving_avs):
    all_mice[i, :num_trials[i]] = mouse_data

In [6]:
fig, axs = plt.subplots()
[axs.plot(m, alpha=0.5, c='gray') for m in moving_avs]
axs.plot(np.nanmean(all_mice,axis=0), c='#7FB5B5', lw=3)

axs.set_xlabel('trial number', fontsize=12)
axs.set_ylabel('performance', fontsize=12)
axs.spines['right'].set_visible(False)
axs.spines['top'].set_visible(False)


In [7]:
[plt.plot(m, alpha=0.5) for m in moving_avs]

[[<matplotlib.lines.Line2D at 0x64793d90>],

In [8]:
plt.show()

In [61]:
behavioural_change = {}
behavioural_change['mouse'] = mice
behavioural_change['pre performance'] = pre_pc
behavioural_change['post performance'] = post_pc
behavioural_change_df = pd.DataFrame(behavioural_change)

In [62]:
df_for_plot = behavioural_change_df.set_index('mouse').transpose()

In [63]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, legend=False, color='grey')
    #data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color='grey' )

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [64]:
from utils.plotting import output_significance_stars_from_pval
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']



fig, ax = plt.subplots(figsize=[1.5, 1.5])
francescas_plot(ax, df_for_plot, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.xticks([0, 1], ['Tone', 'WN'])
plt.ylabel('Performance (%)')
plt.tight_layout()
#significance stars
y = df_for_plot.to_numpy().max() + 2
h = 2
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)

pre_data = df_for_plot.T['pre performance']
post_data = df_for_plot.T['post performance']
stat, pval = stats.ttest_rel(pre_data, post_data)
stars = output_significance_stars_from_pval(pval)
ax.text(.5, y+h, stars, ha='center', fontsize=10)


filepath=os.path.join('W:\\paper', '{}_pre_and_post_state_change_behavioural_effect.pdf'.format(site) )
#fig.savefig(filepath)

In [65]:
pre_data = df_for_plot.T['pre performance']
post_data = df_for_plot.T['post performance']
stat, pval = stats.ttest_rel(pre_data, post_data)
#pval *= .5 # multiply by .5 bc doing a one-sided t-test! we have a hypothesis that it's going a certain way 
pval

0.00019880481396597073

In [48]:
pre_data.shape


(13,)

In [46]:
from scipy.stats import shapiro

In [47]:
differences = pre_data - post_data
shapiro(differences)

ShapiroResult(statistic=0.9323903918266296, pvalue=0.3661174774169922)

In [14]:
df_for_plot.T

,pre performance,post performance
mouse,,
SNL_photo21,100.000000,60.000000
SNL_photo22,100.000000,92.307692
SNL_photo26,100.000000,58.620690
SNL_photo37,90.000000,86.206897
SNL_photo43,91.304348,84.615385
SNL_photo70,91.304348,70.967742
SNL_photo28,100.000000,34.482759
SNL_photo30,96.551724,86.666667
SNL_photo31,92.000000,54.545455


In [17]:
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\state_change_data')
nacc_peaks = pd.read_csv(os.path.join(processed_data_dir, 'nacc_peak_sizes.csv'))
tail_peaks = pd.read_csv(os.path.join(processed_data_dir, 'tail_peak_sizes.csv'))

In [18]:
peaks = tail_peaks 

In [19]:
peak_differences = []
performance_differences = []
for i, row in behavioural_change_df.iterrows():
    mouse = row['mouse']
    peak_differences.append((peaks[mouse][1] - peaks[mouse][0])/peaks[mouse][1]*100)
    performance_differences.append((df_for_plot[mouse][1] - df_for_plot[mouse][0])/df_for_plot[mouse][1]*100)

In [20]:
fig, axs = plt.subplots()
axs.scatter(performance_differences, peak_differences)
axs.set_ylabel('% change in post-pre peak z-score ')
axs.set_xlabel('% difference in performance')


Text(0.5, 0, '% difference in performance')

In [174]:
(peaks[mouse][1] - peaks[mouse][0])

0.5066578094709958

In [175]:
(3-4)/5

-0.2

In [176]:
peaks

,trial type,SNL_photo21,SNL_photo22,SNL_photo26
0,pre,0.266602,0.896028,0.780225
1,post,0.574251,0.894707,1.286883
